# A small notebook for some basic cleaning functions for NLP related tasks. Feel free to edit

My own means of loading in a dataset


In [1]:
from google.colab import drive

drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


# Some libraries

In [2]:
import numpy as np
import pandas as pd
import time
import nltk

## These might be necessary if you are using the stopwords removal function
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('words')



# I kinda use this just to check how fast some of these functions are, you can ignore these
# start_time = time.time()
# print("time elapsed:")
# print("--- %s seconds ---" % (time.time() - start_time))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

### Instantiate your dataset (STARTING POINT)


In [10]:
# My own way, feel free to load in another manner!
data = pd.read_csv('/content/gdrive/My Drive/cds_stuffs/base_no_duplicates.csv', index_col=0)

# Your own dataframe
tweettweet = pd.DataFrame(data)
tweettweet


,tweet_id,label,text,created_at,items,is_reply,contains_url,hashtag,tweetos
0,850490912954350976,abusive,Alex Brosas another idiot #ALDUBKSGoesToUS,42832.979271,"{'hashtags': [{'text': 'ALDUBKSGoesToUS', 'ind...",0,1,ALDUBKSGoesToUS,other
1,848791766853667968,abusive,"as Nancy Reagan would say, 'just say FUCKING N...",42828.290521,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,1,None,other
2,850433664890544000,hateful,I hate er chase because if the Bitch that work...,42832.821296,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,None,other
3,850411934205844992,abusive,SICK OF BITCHES ON THE INTERNET #fbloggers #fb...,42832.761331,"{'hashtags': [{'text': 'fbloggers', 'indices':...",1,1,"fbloggers, fblchat, fbloggers, fblchat",other
4,850100742035836032,hateful,Pineapples do not belong on pizza. Y'all nigga...,42831.902604,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,0,None,other
...,...,...,...,...,...,...,...,...,...
57385,849805798540816000,abusive,just want ONE fucking person JUST ONE,42831.088715,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,0,None,other
57386,849352574654349056,normal,"For fans of Simon R. Green - Fast-paced, grues...",42829.838056,"{'hashtags': [{'text': 'ewgc', 'indices': [122...",0,0,ewgc,other
57387,849736257001463040,abusive,I'm like 5 bad days away from saying fuck it a...,42830.896817,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,0,None,other
57388,848944263366934016,abusive,"""Chick-fil-a's owner is a Trump supporter!"" Me...",42828.711331,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,1,None,other


# Phase 1: Data manipulation

### Drop unneeded columns for training, group up the classes

In [0]:
# start_time = time.time()

### Drop created_at, items, is_reply, contains_url, hashtag, tweetos
tweettweet = tweettweet.drop(columns=['created_at', 'items', 'is_reply', 'contains_url', 'hashtag', 'tweetos'])

### Convert 'abusive' and 'hateful' into class 1, 'normal' and 'spam' into class 0

label = np.array(tweettweet['label'])

label = np.isin(label, ['abusive', 'hateful'])

tweettweet['label'] = label + 0

# print("time elapsed:")
# print("--- %s seconds ---" % (time.time() - start_time))

In [5]:
tweettweet

,tweet_id,label,text
0,850490912954350976,1,Alex Brosas another idiot #ALDUBKSGoesToUS
1,848791766853667968,1,"as Nancy Reagan would say, 'just say FUCKING N..."
2,850433664890544000,1,I hate er chase because if the Bitch that work...
3,850411934205844992,1,SICK OF BITCHES ON THE INTERNET #fbloggers #fb...
4,850100742035836032,1,Pineapples do not belong on pizza. Y'all nigga...
...,...,...,...
57385,849805798540816000,1,just want ONE fucking person JUST ONE
57386,849352574654349056,0,"For fans of Simon R. Green - Fast-paced, grues..."
57387,849736257001463040,1,I'm like 5 bad days away from saying fuck it a...
57388,848944263366934016,1,"""Chick-fil-a's owner is a Trump supporter!"" Me..."


### Unique values counter

In [0]:
## Returns a dictionary of unique values with their counts for a specified column
## So you can convert that column into an array first before passing it in
def unique_counts(array):
  unique, counts = np.unique(array, return_counts=True)
  uc_dict = dict(zip(unique, counts))

  return uc_dict

### Balances your dataset according to a specified ratio(this function was for binary. Feel free to upgrade it)

In [0]:
### Function to remove rows at random from class 0 to match class 1 in the ratio you want
# E.g you want class 0 to have 1.3 times the number of instances in class 1, you set ratio to 1.3

def balance_dataset(dataset, label_col, class_dict, ratio):
  # label_col : the name used for the column containing your labels
  # class_dict : use the unique_counts function to produce it then pass it here
  # ratio : basically the percentage of class 1 that you will reduce class 0 to

  # take the counts 
  c_0_counts = class_dict[0] # class 0 counts
  c_1_counts = class_dict[1] # class 1 counts

  # number of rows in class 0 to drop randomly 
  n = int(c_0_counts - (ratio * c_1_counts))

  # drop the random number of rows
  df_dropped = dataset.drop(dataset[dataset[label_col] == 0].sample(n=n).index)

  # reorder the index
  df_dropped = df_dropped.reset_index()

  # drop previous index row
  df_dropped = df_dropped.drop(columns='index')

  return df_dropped

# Testing Section 0: Test out the first few functions here

In [12]:
class_dict = unique_counts(tweettweet['label'])

tweettweet = balance_dataset(tweettweet, 'label', class_dict, 1)
tweettweet

,tweet_id,label,text
0,850490912954350976,1,Alex Brosas another idiot #ALDUBKSGoesToUS
1,848791766853667968,1,"as Nancy Reagan would say, 'just say FUCKING N..."
2,850433664890544000,1,I hate er chase because if the Bitch that work...
3,850411934205844992,1,SICK OF BITCHES ON THE INTERNET #fbloggers #fb...
4,850100742035836032,1,Pineapples do not belong on pizza. Y'all nigga...
...,...,...,...
25187,850744597030592000,1,Plot twist: he never stopped fucking with her ...
25188,849805798540816000,1,just want ONE fucking person JUST ONE
25189,849736257001463040,1,I'm like 5 bad days away from saying fuck it a...
25190,848944263366934016,1,"""Chick-fil-a's owner is a Trump supporter!"" Me..."


Just a means for me to check if the counts were done correctly

In [13]:
c1 = 0 # class 1 counter
c0 = 0 # class 0 counter
for x in tweettweet['label']:
  if x == 1:
    c1 += 1
  else:
    c0 += 1
print(c1)
print(c0)


12596
12596


# Remove duplicates from base dataset by column

First get a dictionary of unique terms and their counts in the dictionary, then extract out the terms that have more than 1 count 

In [0]:
### get the duplicate terms and their counts in the column
def duplicate_counts(dataset, col_name):
  # col_name : name of the column you want to check for duplicates

  # change the dataframe column to np array
  col_np = np.array(dataset[col_name].astype('str')) 

  # using unique counts function to get the dictionary of unique terms and counts
  uni_dict = unique_counts(col_np)

  # instantiate the dictionary that will contain the duplicates
  dup_dict = {}

  for key in uni_dict:
    if uni_dict[key] > 1:
      dup_dict[key] = uni_dict[key]

  return dup_dict

57390


### Function for removing rows that contain duplicates and resetting the index

In [0]:
## TO REMOVE DUPLICATES FROM A COLUMN 
def removeduplicates(dataset, col_name, dup_dict):
  # dataset : your dataset
  # col_name : name of the column you want to remove duplicates from
  # dup_dict : the dictionary of duplicates you get from using the previous function 'duplicate_counts'

  counter = 0 # for every 
  for index, x in enumerate(dataset[col_name]):
    for key in dup_dict:
      # if the word from dataset is equal to the word in dict and value of that word count more than 1, delete row
      if x == key and dup_dict[key] > 1:  
        dataset = dataset.drop(dataset.index[index-counter])
        # minus the count of the for this word in dup dictionary by 1
        dup_dict[key] -= 1
        counter += 1
        # break after finding
        break
    # if x == 2:
  dataset = pd.DataFrame(dataset).reset_index()
  dataset = dataset.drop(columns='index')

  return dataset

In [0]:
# tweettweet

In [0]:
# tweettweet = removeduplicatess(tweettweet, 'text')
# tweettweet

,level_0,index,tweet_id,label,text,created_at,items,is_reply,contains_url,hashtag,tweetos
0,0,0,850490912954350976,abusive,Alex Brosas another idiot #ALDUBKSGoesToUS,42832.979271,"{'hashtags': [{'text': 'ALDUBKSGoesToUS', 'ind...",0,1,ALDUBKSGoesToUS,other
1,1,1,848791766853667968,abusive,"as Nancy Reagan would say, 'just say FUCKING N...",42828.290521,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,1,None,other
2,3,3,850433664890544000,hateful,I hate er chase because if the Bitch that work...,42832.821296,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,None,other
3,5,5,850411934205844992,abusive,SICK OF BITCHES ON THE INTERNET #fbloggers #fb...,42832.761331,"{'hashtags': [{'text': 'fbloggers', 'indices':...",1,1,"fbloggers, fblchat, fbloggers, fblchat",other
4,6,6,850100742035836032,hateful,Pineapples do not belong on pizza. Y'all nigga...,42831.902604,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,0,None,other
...,...,...,...,...,...,...,...,...,...,...,...
57385,62549,62573,849805798540816000,abusive,just want ONE fucking person JUST ONE,42831.088715,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,0,None,other
57386,62550,62574,849352574654349056,normal,"For fans of Simon R. Green - Fast-paced, grues...",42829.838056,"{'hashtags': [{'text': 'ewgc', 'indices': [122...",0,0,ewgc,other
57387,62551,62575,849736257001463040,abusive,I'm like 5 bad days away from saying fuck it a...,42830.896817,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,0,None,other
57388,62552,62576,848944263366934016,abusive,"""Chick-fil-a's owner is a Trump supporter!"" Me...",42828.711331,"{'hashtags': [], 'symbols': [], 'user_mentions...",1,1,None,other


In [0]:
from google.colab import files
# exportnow.to_csv('base_no_duplicates.csv') 
tweettweet.to_csv('/content/gdrive/My Drive/cds_stuffs/normal_has_more_1_4.csv')

# Phase 2: Data Preprocessing/Cleaning 

Note: Choose whichever function you need. For how I did it, I always run lowercase function first as it contains my stringify functionality. You can always do it beforehand!

Note(again): For majority of these functions, you will only have dataset and text_col_name as the parameters, where text_col_name is just the name of the column where your text resides. For special cases, check comments

## Basic preprocessing functions

### Lower case and stringify

In [0]:
def lower_case(dataset, text_col_name):
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join(x.lower() for x in str(x).split()))

  return dataset

### Remove hashtag

In [0]:
def remove_hashtag(dataset, text_col_name):
  dataset[text_col_name] = dataset[text_col_name].str.replace('#[\w]*','')

  return dataset

### Remove punctuation

In [0]:
def remove_punctuation(dataset, text_col_name):
  dataset[text_col_name] = dataset[text_col_name].str.replace('[^\w\s]','')

  return dataset

### Stopword removal

In [0]:
from nltk.corpus import stopwords

def remove_stops(dataset, text_col_name):
  stops = stopwords.words('english')
  
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join(x for x in x.split() if x not in stops))

  return dataset

### Non English removal

In [0]:
import nltk

def remove_nonenglish(dataset, text_col_name):

  words = set(nltk.corpus.words.words())

  # dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \ if w.lower() in words or not w.isalpha()))
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join(x for x in x.split() if x in words))

  return dataset

### Common word removal

In [0]:
# note, must make sure the words are in string form. so run the lowercase function first

def remove_commonwords(dataset, text_col_name, number_to_remove):
  # number_to_remove : how many common words you want to remove from the whole dataset. eg if 'here' is a common word, it is removed from every line

  n = number_to_remove

  ## get the frequency of each word
  word_frequency = pd.Series(' '.join(dataset[text_col_name]).split()).value_counts()[:n]

  ## remove the n most common words
  word_frequency = list(word_frequency.index)
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join(x for x in x.split() if x not in word_frequency))

  return dataset

### Rare word removal

In [0]:
def remove_rarewords(dataset, text_col_name, number_to_remove):
  # number_to_remove : how many common words you want to remove from the whole dataset. eg if 'here' is a common word, it is removed from every line

  n = number_to_remove

  ## get the frequency of each word
  word_frequency = pd.Series(' '.join(dataset[text_col_name]).split()).value_counts()[-n:]

  ## remove the n most common words
  word_frequency = list(word_frequency.index)
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join(x for x in x.split() if x not in word_frequency))

  return dataset

### Spelling correction (avoid this first)

In [0]:
# ## warning this can be freaking long so keep in mind to test out a few terms first (eg)

# from textblob import TextBlob

# def spell_correct(dataset, text_col_name, n_terms):
#   n = n_terms
#   dataset[text_col_name][:n].apply(lambda x: str(TextBlob(x).correct()))

#   return dataset

# Testing Section 1

In [0]:
# made a copy to make life easier
testing_base = tweettweet.copy()
testing_base

,tweet_id,label,text
0,850490912954350976,1,Alex Brosas another idiot #ALDUBKSGoesToUS
1,848791766853667968,1,"as Nancy Reagan would say, 'just say FUCKING N..."
2,850433664890544000,1,I hate er chase because if the Bitch that work...
3,850411934205844992,1,SICK OF BITCHES ON THE INTERNET #fbloggers #fb...
4,850100742035836032,1,Pineapples do not belong on pizza. Y'all nigga...
...,...,...,...
30226,850744597030592000,1,Plot twist: he never stopped fucking with her ...
30227,849805798540816000,1,just want ONE fucking person JUST ONE
30228,849736257001463040,1,I'm like 5 bad days away from saying fuck it a...
30229,848944263366934016,1,"""Chick-fil-a's owner is a Trump supporter!"" Me..."


In [0]:
### Testing Section


testing_base = remove_hashtag(testing_base, 'text')
testing_base = lower_case(testing_base, 'text')
testing_base = remove_punctuation(testing_base, 'text')
testing_base = remove_stops(testing_base, 'text')

# testing_base = spell_correct(testing_base, 'text', len(testing_base))
# testing_base



# freq = remove_commonwords(testing_base, 'text', 10)
# freq
testing_base

,tweet_id,label,text
0,850490912954350976,1,alex brosas another idiot
1,848791766853667968,1,nancy reagan would say say fucking something like
2,850433664890544000,1,hate er chase bitch works shes literally evil
3,850411934205844992,1,sick bitches internet
4,850100742035836032,1,pineapples belong pizza yall niggas nasty
...,...,...,...
30226,850744597030592000,1,plot twist never stopped fucking ðÿœš
30227,849805798540816000,1,want one fucking person one
30228,849736257001463040,1,im like 5 bad days away saying fuck running bu...
30229,848944263366934016,1,chickfilas owner trump supporter damn dats crazy


## Slightly advanced preprocessing functions

### Tokenization

In [0]:
def tokenize(dataset, text_col_name):
  
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: TextBlob(x).words)

  return dataset


### Stemming

In [0]:
from nltk.stem import PorterStemmer

def stem(dataset, text_col_name):
  plant = PorterStemmer()
  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join([plant.stem(word) for word in x.split]))

  return dataset


### Lemmatization

In [0]:
from textblob import Word

def lemma(dataset, text_col_name):

  dataset[text_col_name] = dataset[text_col_name].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

  return dataset


### Sentiment Analysis 

In [0]:
def get_sentiment(dataset, text_col_name):

  dataset['sentiment'] = dataset[text_col_name].apply(lambda x: TextBlob(x).sentiment[0])

  return dataset


In [0]:
testing_base2 = testing_base.copy()

# Testing Section 2

In [0]:
# testing_base2 = tokenize(testing_base2,'text', 1)

testing_base2 = lemma(testing_base2,'text')

# testing_base2 = get_sentiment(testing_base2, 'text')
# 
# testing_base2 = remove_nonenglish(testing_base, 'text')

testing_base2

,tweet_id,label,text
0,850490912954350976,1,alex brosas another idiot
1,848791766853667968,1,nancy reagan would say say fucking something like
2,850433664890544000,1,hate er chase bitch work shes literally evil
3,850411934205844992,1,sick bitch internet
4,850100742035836032,1,pineapple belong pizza yall nigga nasty
...,...,...,...
30226,850744597030592000,1,plot twist never stopped fucking ðÿœš
30227,849805798540816000,1,want one fucking person one
30228,849736257001463040,1,im like 5 bad day away saying fuck running bud...
30229,848944263366934016,1,chickfilas owner trump supporter damn dat crazy


## My own means of getting my processed dataset output to gdrive

In [0]:
# <your_dataset_object_name>.to_csv(<your_preferred_file_path.csv>)    dont forget your ''
testing_base2.to_csv('/content/gdrive/My Drive/cds_stuffs/normal_has_more_1_4.csv')
